In [1]:
import openai, os

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [5]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":200,
    "stop":"}",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()


In [6]:
query('1+1=')

'2\n\n2+2=4'